In [1]:
import json
import sys
sys.path.append('/home/nbuser/library/')

import pandas as pd
import numpy as np
import requests
import key


In [2]:
def get_pop(x,y):
    
    
    url = 'https://koordinates.com/services/query/v1/vector.json'
    
    params = {
        'key' : key.key,
        'layer' : 104612,
        'x' : x,
        'y' : y
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        return response.status_code
    
    pop = response.json()['vectorQuery']['layers']['104612']['features'][0]['properties']['C18_CURPop']
    
    return pop

In [3]:
get_pop(174.9040694,-37.0129205)

174

In [12]:
df = pd.read_csv('Dataset for Assignment.csv')

In [14]:
pd.options.mode.chained_assignment = None  # default='warn'
#df['C18_CURPop'] = np.vectorize(get_pop)(df['Longitude'], df['Latitude'])
df['C18_CURPop'] = df.apply(lambda row: get_pop(row['Longitude'], row['Latitude']), axis = 1)

In [15]:
df.head()

,Bedrooms,Bathrooms,Address,Land area,CV,Latitude,Longitude,SA1,0-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60+ years,Suburbs,C18_CURPop
0,5,3.0,"106 Lawrence Crescent Hill Park, Auckland",714,960000,-37.012920,174.904069,7009770,48,27,24,21,24,21,Manurewa,174
1,5,3.0,"8 Corsica Way Karaka, Auckland",564,1250000,-37.063672,174.922912,7009991,42,18,12,21,15,30,Karaka,129
2,6,4.0,"243 Harbourside Drive Karaka, Auckland",626,1250000,-37.063580,174.924044,7009991,42,18,12,21,15,30,Karaka,129
3,2,1.0,"2/30 Hardington Street Onehunga, Auckland",65,740000,-36.912996,174.787425,7007871,42,6,21,21,12,15,Onehunga,120
4,3,1.0,"59 Israel Avenue Clover Park, Auckland",601,630000,-36.979037,174.892612,7008902,93,27,33,30,21,33,Clover Park,231


In [16]:
df.to_csv('Dataset_1.csv', index=False)

In [19]:
!curl https://www.otago.ac.nz/wellington/otago730395.xlsx -O

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1178k  100 1178k    0     0   359k      0  0:00:03  0:00:03 --:--:--  359k


In [56]:
df_dep = pd.read_excel('otago730395.xlsx')
df_dep= depdf.rename(columns={'SA12018_code' : 'SA1'})
df_dep.head()

,SA1,NZDep2018,NZDep2018_Score,URPopnSA1_2018,SA22018_code,SA22018_name
0,7000000,10.0,1245.0,141,100100,North Cape
1,7000001,10.0,1245.0,114,100100,North Cape
2,7000002,NaN,NaN,0,100300,Inlets Far North District
3,7000003,10.0,1207.0,225,100100,North Cape
4,7000004,9.0,1093.0,138,100100,North Cape


In [57]:
df_final = df.merge(df_dep[['SA1', 'NZDep2018']])

In [68]:
df_final.head()

,Bedrooms,Bathrooms,Address,Land area,CV,Latitude,Longitude,SA1,0-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60+ years,Suburbs,Population,Deprivation Index
0,5,3.0,"106 Lawrence Crescent Hill Park, Auckland",714,960000,-37.012920,174.904069,7009770,48,27,24,21,24,21,Manurewa,174,6.0
1,5,3.0,"8 Corsica Way Karaka, Auckland",564,1250000,-37.063672,174.922912,7009991,42,18,12,21,15,30,Karaka,129,1.0
2,6,4.0,"243 Harbourside Drive Karaka, Auckland",626,1250000,-37.063580,174.924044,7009991,42,18,12,21,15,30,Karaka,129,1.0
3,2,1.0,"2/30 Hardington Street Onehunga, Auckland",65,740000,-36.912996,174.787425,7007871,42,6,21,21,12,15,Onehunga,120,2.0
4,3,1.0,"59 Israel Avenue Clover Park, Auckland",601,630000,-36.979037,174.892612,7008902,93,27,33,30,21,33,Clover Park,231,9.0


In [67]:
df_final = df_final.rename(columns={'C18_CURPop' : 'Population'})
df_final= df_final.rename(columns={'NZDep2018' : 'Deprivation Index'})

In [69]:
df_final.to_csv('Dataset_Final.csv', index=False)